In [1]:
import tensorflow as tf

import time
import os
import shutil

/home/cppxaxa/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cppxaxa/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cppxaxa/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cppxaxa/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
# https://stackoverflow.com/questions/33997823/tensorflow-mlp-not-training-xor

# create a placeholder for the input
# None indicates a variable batch size for the input
n_input = tf.placeholder(tf.float32, shape=[None, 2], name="n_input")
n_output = tf.placeholder(tf.float32, shape=[None, 1], name="n_output")

################
# hidden layer #
################

layer1_count = 5
layer1_b_hidden = tf.Variable(tf.random_normal([layer1_count]), name="layer1_hidden_bias")
layer1_W_hidden = tf.Variable(tf.random_normal([2, layer1_count]), name="layer1_hidden_weights")
layer1 = tf.sigmoid(tf.matmul(n_input, layer1_W_hidden) + layer1_b_hidden)

################
# output layer #
################

W_output = tf.Variable(tf.random_normal([layer1_count, 1]), name="output_weights")  # output layer's weight matrix
output = tf.sigmoid(tf.matmul(layer1, W_output))  # calc output layer's activation
result = tf.identity(output, name="result")

############
# learning #
############

cross_entropy = -(n_output * tf.log(output) + (1 - n_output) * tf.log(1 - output))
# error = cross_entropy
error = tf.square(n_output - output)  # simpler, but also works

loss = tf.reduce_mean(error)  # mean the cross_entropy
optimizer = tf.train.AdamOptimizer(0.01)  # take a gradient descent for optimizing with a "stepsize" of 0.1
train = optimizer.minimize(loss)  # let the optimizer train

In [4]:
# https://stackoverflow.com/questions/33997823/tensorflow-mlp-not-training-xor
# https://stackoverflow.com/a/47235448/6484802 - Signatures

##############
# checkpoint #
##############

model_path = 'xor_010_ck'
if os.path.exists(model_path):
    shutil.rmtree(model_path)
    time.sleep(0.5)

saver = tf.saved_model.builder.SavedModelBuilder(model_path)

####################
# initialize graph #
####################
init = tf.global_variables_initializer()

sess = tf.Session()  # create the session and therefore the graph
sess.run(init)  # initialize all variables  


###########
# summary #
###########

writer = tf.summary.FileWriter("xor_010", sess.graph)
cost_summary_entity = tf.summary.scalar('loss', loss)
fetch_summary = tf.summary.merge_all()

#####################
# train the network #
#####################

# define input and output data
input_data = [[0., 0.], [0., 1.], [1., 0.], [1., 1.]]  # XOR input
output_data = [[0.], [1.], [1.], [0.]]  # XOR output

batch_xdata = [
    [[0.19, 0.05]], [[0.04, 0.964]], [[0.956, 0.03]], [[0.899, 0.987]],
    [[0.2, 0.89]], [[0.98, 0.91]], [[0.23, 0.01]], [[0.28, 0.99]],
    [[0.93, 0.92]], [[0.88, 0.025]], [[0.18, 0.97]], [[0.11, 0.08]],
    [[0.95, 0.09]], [[0.977, 0.991]], [[0.061, 0.091]], [[0.12, 0.955]]
]
batch_ydata = [
    [[0.]], [[1.]], [[1.]], [[0.]],
    [[1.]], [[0.]], [[0.]], [[1.]],
    [[0.]], [[1.]], [[1.]], [[0.]],
    [[1.]], [[0.]], [[0.]], [[1.]]
]

bak_epoch = 0
t_start = time.clock()
for epoch in range(0, 5000):
    bak_epoch = epoch
    # run the training operation
    for i in range(len(batch_xdata)):
        xdata = batch_xdata[i]
        ydata = batch_ydata[i]
        cvalues = sess.run([train, loss],
                           feed_dict={n_input: xdata, n_output: ydata})

    # print some debug
    if epoch % 50 == 0:
        summary_data = sess.run(fetch_summary, feed_dict={n_input: input_data, n_output: output_data})
        writer.add_summary(summary_data, epoch)
    
    if epoch % 200 == 0:
        loss_value = cvalues[1]
        if loss_value < 0.002:
            print("loss: {}".format(loss_value))
            break
            
    if epoch % 1000 == 0:
        print("")
        print("step: {:>3}".format(epoch))
        print("loss: {}".format(loss_value))
        
t_end = time.clock()

tensor_info_x = tf.saved_model.utils.build_tensor_info(n_input)
tensor_info_y = tf.saved_model.utils.build_tensor_info(result)

prediction_signature = (
    tf.saved_model.signature_def_utils.build_signature_def(
        inputs={'x_input': tensor_info_x},
        outputs={'y_output': tensor_info_y},
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

saver.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
        prediction_signature 
    })
saver.save()

print("")
print('Elapsed time ', t_end - t_start)
print("Epoch", bak_epoch)
for el in input_data:
    print("input: {} | output: {}".format(el, sess.run(output, feed_dict={n_input: [el]})))
    
sess.close()
writer.close()


step:   0
loss: 0.4670673906803131
loss: 0.0002204965567216277
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: xor_010_ck/saved_model.pb

Elapsed time  8.042747
Epoch 400
input: [0.0, 0.0] | output: [[0.01750605]]
input: [0.0, 1.0] | output: [[0.9891367]]
input: [1.0, 0.0] | output: [[0.97706455]]
input: [1.0, 1.0] | output: [[0.02910046]]
